In [1]:
import curses
from curses import KEY_RIGHT, KEY_LEFT, KEY_UP, KEY_DOWN
from random import randint
import tensorflow as tf
from tensorflow import layers
import numpy as np
import os
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.estimator import regression
import tflearn

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [2]:
def training_model():
    network = input_data(shape=[None, 3, 1], name='input')
    network = fully_connected(network, 1, activation='linear')
    network = regression(network, optimizer='adam', learning_rate=1e-2, loss='mean_square', name='target')
    model = tflearn.DNN(network, tensorboard_dir='log')
    return model



In [3]:
training_data = []

In [4]:
# left 0 right 1 up 2 down 3
# direction, have obstacle, will die?, decision
training_data.append([np.array([0, 1, 1, 0]), np.array(0)])
training_data.append([np.array([0, 1, 0, 2]), np.array(2)])
training_data.append([np.array([2, 1, 0, 0]), np.array(0)])
training_data.append([np.array([3, 0, 0, 3]), np.array(3)])

In [5]:
list((i[1] for i in training_data))

[array(0), array(2), array(0), array(3)]

In [6]:
x = np.array([i[0] for i in training_data]).reshape(-1, 4, 1)
y = np.array([i[1] for i in training_data]).reshape(-1, 1)
model = training_model()
model.fit(x, y, n_epoch=10, shuffle=True)

---------------------------------
Run id: OPHKRG
Log directory: log/
---------------------------------
Training samples: 4
Validation samples: 0
--
Training Step: 1  | time: 0.093s
| Adam | epoch: 001 | loss: 0.00000 -- iter: 4/4
--
Training Step: 2  | total loss: 2.78219 | time: 0.002s
| Adam | epoch: 002 | loss: 2.78219 -- iter: 4/4
--
Training Step: 3  | total loss: 2.92246 | time: 0.003s
| Adam | epoch: 003 | loss: 2.92246 -- iter: 4/4
--
Training Step: 4  | total loss: 2.84565 | time: 0.003s
| Adam | epoch: 004 | loss: 2.84565 -- iter: 4/4
--
Training Step: 5  | total loss: 2.73829 | time: 0.003s
| Adam | epoch: 005 | loss: 2.73829 -- iter: 4/4
--
Training Step: 6  | total loss: 2.62703 | time: 0.003s
| Adam | epoch: 006 | loss: 2.62703 -- iter: 4/4
--
Training Step: 7  | total loss: 2.51722 | time: 0.005s
| Adam | epoch: 007 | loss: 2.51722 -- iter: 4/4
--
Training Step: 8  | total loss: 2.41017 | time: 0.003s
| Adam | epoch: 008 | loss: 2.41017 -- iter: 4/4
--
Training Step: 9  

In [50]:
# left 0 right 1 up 2 down 3
# direction, have obstacle, will die?, decision
best_fit = {}
for action in range(0,3):
    best_fit.update({action: model.predict(np.array([0, 1, 1, action]).reshape(-1, 4, 1))})
best_fit
import operator
max(best_fit.iteritems(), key=operator.itemgetter(1))[0]

2

In [24]:
tf.reset_default_graph()
network = input_data(shape=[None, 3, 1], name='input')
network = fully_connected(network, 25, activation='relu')
network = fully_connected(network, 25, activation='relu')
network = fully_connected(network, 1, activation='linear')
network = regression(network, optimizer='adam', learning_rate=1e-2, loss='mean_square', name='target')
model_load = tflearn.DNN(network, tensorboard_dir='log')
model_load.load('snake_bot.h5')
# left 0 right 1 up 2 down 3
# direction, have obstacle, decision
import operator
KEYS = [KEY_LEFT, KEY_RIGHT, KEY_UP, KEY_DOWN]
results = []
best_fit = {}
r = model_load.evaluate(np.array([int(258), int(258), 1]).reshape(-1, 3, 1), np.array([1]).reshape(-1, 1))
print(r)
for action in range(0,5):
    best_fit.update({action: model_load.predict(np.array([int(0), int(action), 1]).reshape(-1, 3, 1))})
key_index = min(best_fit.iteritems(), key=operator.itemgetter(1))[0]
print(best_fit)
print(key_index)
print(KEYS[key_index])

INFO:tensorflow:Restoring parameters from /home/alex/Development/snake-game/snake_bot.h5
[1.0]
{0: array([[1.0000453]], dtype=float32), 1: array([[1.0039898]], dtype=float32), 2: array([[1.0079341]], dtype=float32), 3: array([[1.0118785]], dtype=float32), 4: array([[1.015823]], dtype=float32)}
0
260


In [4]:
import curses
from curses import KEY_RIGHT, KEY_LEFT, KEY_UP, KEY_DOWN
from random import randint
import tensorflow as tf
from tensorflow import layers
import numpy as np
import os
from keras.models import load_model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.estimator import regression
import tflearn
model = load_model('snake_bot.h5')

In [37]:
results = []
for k in range(0,4):
    results.append(model.predict(np.array([3,k,1]).reshape(-1,3))) #.reshape(-1, 3, 1)))
    #results.append(model.predict_classes(np.array([0,k,1]).reshape(-1,3))) #.reshape(-1, 3, 1)))
results

[array([[1.0151893]], dtype=float32),
 array([[1.1374441]], dtype=float32),
 array([[1.2096672]], dtype=float32),
 array([[1.3328403]], dtype=float32)]

In [19]:
model.predict_classes(np.array([3,3,1]).reshape(-1,3))

array([[1]], dtype=int32)

In [49]:
import operator
best_fit = {}
for action in range(0,4):
    best_fit.update({action: model.predict(np.array([0, action, 1]).reshape(-1, 3))})
key_index = max(best_fit.iteritems(), key=operator.itemgetter(1))[0]
print best_fit
print(key_index)

{0: array([[1.1552042]], dtype=float32), 1: array([[1.2601165]], dtype=float32), 2: array([[1.2521561]], dtype=float32), 3: array([[1.2549958]], dtype=float32)}
1
